**Run this cell first!**

In [ ]:
# Always run this code.
%config InteractiveShell.ast_node_interactivity="none"
import sys
if 'google.colab' in sys.modules:
    !pip install --force-reinstall git+https://github.com/jamcoders/jamcoders-public-2023.git --quiet
    !pip install networkx==2.6.3 --quiet
from jamcoders.base_utils import *
import networkx as nx
from jamcoders.week4.labw4d1b import *

# Week 4 Day 1B, Graph Search


# 0. Recap of Paths in Graph

#### You may refer to answers in the previous notebook but make sure you understand the functions

### 0.1 get_neighbors()

Write a function that takes in an undirected graph `G` and a node `n`, and `return`s a list of all the neighbors of `n`. If `n` has no neighbors, `return` the empty list `[]`.

In [ ]:
def get_neighbors(G, n):
    """ returns the list of neighbors of node n in graph G
    Inputs:
        G: The graph represented as an adjacency list
            type: list(list(int))
        n: The node
            type: string or int
    Returns: The list of neighbors
            type: list[string or int]
    """

In [ ]:
# Test cases: (do not change this code!)
G = nx.Graph()
G.add_nodes_from(range(5))
G.add_edges_from([(0, 1), (1, 2), (0, 2), (2, 3)])
pos = nx.circular_layout(G)
nx.draw_networkx(G,pos=pos, arrows=True, arrowsize=10, with_labels=True, **options)
graph = nx.to_dict_of_lists(G)

assert_equal(want=set([1, 2]), got=set(get_neighbors(graph, 0)))
assert_equal(want=set([0, 1, 3]), got=set(get_neighbors(graph, 2)))
assert_equal(want=set([]) , got=set(get_neighbors(graph, 4)))

### 0.2 is_neighboring()

You are given an undirected graph `G` and two nodes `n1` and `n2`. Check if there is a edge from `n1` to `n2`.

In [ ]:
def is_neighboring(G, n1, n2):
    """ returns True if there is an edge from n1 to n2 in G
    Inputs:
        G: The graph
            type: dict{list[string or int]}
        n1: A node
            type: string or int
        n2: Another node
            type: string or int
    Returns:
            type: bool
    """


In [ ]:
# Test cases:
G = nx.Graph()
G.add_nodes_from(range(5))
G.add_edges_from([(0, 1), (1, 2), (0, 2), (2, 3)])
pos = nx.circular_layout(G)
nx.draw_networkx(G,pos=pos, arrows=True, arrowsize=10, with_labels=True, **options)
graph = nx.to_dict_of_lists(G)

print(f'The graph G is: {graph}')
print()

assert_equal(want=False, got=is_neighboring(graph, 0, 3))
assert_equal(want=True , got=is_neighboring(graph, 2, 3))
assert_equal(want=False,got=is_neighboring(graph, 4, 1))
assert_equal(want=True, got=is_neighboring(graph, 2, 1))

### 0.3 path_2()

#### Note: This question is a bit different from last week's, which asked for a path of length <= 2. This question wants a path of exactly 2

You are given a **directed** graph `G` and two nodes `n1` and `n2`. Check if there is an path of **exactly** length `2`  that starts from `n1` and ends at `n2`.

For example in the graph above, `1 -> 2 -> 3` is a path of length `2`, so `path_2(G, 1, 3)` should return `True`. `2 -> 3` is a path with one edge, so `path_2(G, 2, 3)` should return `False`. There is no path to `4`, so `path_2(G,  3, 4)` should return `False`.


In [ ]:
def path_2(G, n1, n2):
    """ returns True if there is a path of length 2 or smaller from n1 to n2 in G
    Inputs:
        G: The graph
            type: list(list(int))
        n1: A node
            type: int
        n2: Another node
            type: int
    Returns:
            type: bool
    """


In [ ]:
# Test cases:
G = nx.DiGraph()
n = 10
G.add_nodes_from(range(n))
edges = []
for i in range(n - 1):
    for j in range(i + 1, n):
        if j <= i ** 2 and j % 2 == i % 2:
            edges.append((i, j))
G.add_edges_from(edges)
pos = nx.circular_layout(G)
nx.draw_networkx(G,pos=pos, arrows=True, arrowsize=10, with_labels=True, **options)
graph = nx.to_dict_of_lists(G)
print(f'The graph G is: {graph}')
print()

assert_equal(want=False, got=path_2(graph, 1, 4))
assert_equal(want=False, got=path_2(graph, 7, 3))
assert_equal(want=True, got=path_2(graph, 3, 7))
assert_equal(want=True, got=path_2(graph, 2, 8))
assert_equal(want=False, got=path_2(graph, 4, 6))
assert_equal(want=True, got=path_2(graph, 2, 6))
assert_equal(want=False, got=path_2(graph, 8, 1))

# 1. Depth-First Search

We've just learned some important concepts on graphs, such as neighbors and paths. Another important concept is searching a graph. One algorithm to search a graph is Depth-First Search (DFS), and is often used to test if two vertices have a path between each other (i.e. they are *connected*).

In the last exercise, we figured out how to test if two vertices had a path of length `2`. Let's figure out how to test if two vertices had a path of length `3`. From there, we will test if two vertices have a path of arbitrary length.

### 1.0 Warmup

If there is a path from `x` to `y` of **any length**, set `connected_x_y = True`. Otherwise set `connected_x_y = False`.

In [ ]:
connected_1_4 = ...
connected_7_3 = ...
connected_3_7 = ...
connected_2_8 = ...
connected_4_6 = ...
connected_2_6 = ...
connected_8_1 = ...

check_answer_1_0([connected_1_4, connected_7_3,connected_3_7,connected_2_8,connected_4_6,connected_2_6,connected_8_1])

### 1.2 path_3()

You are given a **directed** graph `G` and two nodes `n1` and `n2`. Check if there is a path of **exactly** length `3`  that starts from `n1` and ends at `n2`.

For example in the graph in Question 0.3 above, `2 -> 4 -> 6 -> 8` is a path of length `3`, so `path_3(G, 2, 8)` should return `True`. `2 -> 4` is a path with one edge, so `path_3(G, 2, 4)` should return `False`. There is no path to `0`, so `path_2(G, 3, 0)` should return `False`.

You may want to use the functions `get_neighbors()` and `is_neighboring()` in your solution, but you don't have to.

In [ ]:
def get_neighbors(G, n):
    """ returns the list of neighbors of node n in graph G
    Inputs:
        G: The graph
            type: dict{list[string or int]}
        n: The node
            type: string or int
    Returns: The list of neighbors
            type: list[string or int]
    """
    # YOUR CODE HERE (You may copy/paste from last exercise)

def is_neighboring(G, n1, n2):
    """ returns True if there is an edge from n1 to n2 in G
    Inputs:
        G: The graph
            type: dict{list[string or int]}
        n1: A node
            type: string or int
        n2: Another node
            type: string or int
    Returns:
            type: bool
    """
    # YOUR CODE HERE (You may copy/paste from last exercise)

In [ ]:
def path_3(G, n1, n2):
    """ returns True if there is a path of exactly length 3 from n1 to n2 in G
    Inputs:
        G: The graph
            type: list(list(int))
        n1: A node
            type: int
        n2: Another node
            type: int
    Returns:
            type: bool
    """
    # YOUR CODE HERE

In [ ]:
# Test cases:
G = nx.DiGraph()
n = 10
G.add_nodes_from(range(n))
edges = []
for i in range(n - 1):
    for j in range(i + 1, n):
        if j <= i ** 2 and j % 2 == i % 2:
            edges.append((i, j))
G.add_edges_from(edges)
pos = nx.circular_layout(G)
nx.draw_networkx(G,pos=pos, arrows=True, arrowsize=10, with_labels=True, **options)
graph = nx.to_dict_of_lists(G)
print(f'The graph G is: {graph}')
print()

assert_equal(want=False, got=path_3(graph, 1, 4))
assert_equal(want=False, got=path_3(graph, 7, 3))
assert_equal(want=False, got=path_3(graph, 3, 7))
assert_equal(want=True, got=path_3(graph, 2, 8))
assert_equal(want=False, got=path_3(graph, 4, 6))
assert_equal(want=False, got=path_3(graph, 2, 6))
assert_equal(want=False, got=path_3(graph, 8, 1))

### 1.3 is_connected() with DFS

DFS is an algorithm that lets us determine whether there is a path of arbitrary length between two nodes `n1` and `n2`. In general, DFS starts at `n1`, and recurses on the neighbors of `n1`, the neighbors of the neighbors of `n1`, etc.

Suppose we have a recursive function `dfs(v)`, which checks if there's a path from `n1 = 2` to `n2 = 8`. One possible recursive stack trace is the following

|        |        |        |        |
|--------|--------|--------|--------|
| `dfs(2)` |        |        |        |
| `--->`   | `dfs(4)` |        |        |
|        | `--->`   | `dfs(6)` |        |
|        |        | `--->`   | `dfs(8)` |

which indicates that node `8` is reachable from node `2`. We could also use DFS to find the shortest path from `n1 = 3` to `n2 = 9`. However, note that the following trace could occur

|        |        |        |        |
|--------|--------|--------|--------|
| `dfs(3)` |        |        |        |
| `--->` | `dfs(5)` |        |        |
|        | `--->`   | `dfs(7)` |        |
|        |        | `--->`   | `dfs(9)` |
|        | `dfs(5)` |        |        |
|        | `--->`   | `dfs(9)` |        |

where we visit nodes 7 and 9 multiple times. This is inefficient, and could result in infinite recursion if a cycle exists in the graph. We overcome this by keeping track of which vertices have been visited already.

Write a function `is_connected()` to check if there is any path from node `n1` to node `n2` in a directed graph `G`.

#### Pseudocode Hint: (Try doing it before looking at the hint)
* Set the entry in visited that corresponds to `u` to True
* If you have reached the target node `t`, return True
* Iterate over the neighbors of `u`
* If the neighbor has been visited, ignore it! Because you have explored it before
* If the neighbor has not been visited,
    *  Do DFS on the neighbor
    *  If the result of the DFS on the neighbor is True, return True
* After checking all neighbors, if the target has still not been found, return False

In [ ]:
def dfs(u, t, G, visited):
    # YOUR CODE HERE

def is_connected_dfs(G, n1, n2):
    """ returns True if there is a path from n1 to n2 in G
    Inputs:
        G: The graph
            type: list(list(int))
        n1: A node
            type: int
        n2: Another node
            type: int
    Returns:
            type: bool
    """
    visited = [False] * len(G) # visited[i] is True if vertex i has been visited
    return dfs(n1, n2, G, visited)

In [ ]:
# Graph:
G = nx.DiGraph()
n = 10
G.add_nodes_from(range(n))
edges = []
for i in range(n - 1):
    for j in range(i + 1, n):
        if j <= i ** 2 and j % 2 == i % 2:
            edges.append((i, j))
G.add_edges_from(edges)
pos = nx.circular_layout(G)
nx.draw_networkx(G,pos=pos, arrows=True, arrowsize=10, with_labels=True, **options)
graph = nx.to_dict_of_lists(G)

assert_equal(want=False, got=is_connected_dfs(graph, 1, 4))
assert_equal(want=False, got=is_connected_dfs(graph, 7, 3))
assert_equal(want=True, got=is_connected_dfs(graph, 3, 7))
assert_equal(want=True, got=is_connected_dfs(graph, 2, 8))
assert_equal(want=True, got=is_connected_dfs(graph, 4, 6))
assert_equal(want=True, got=is_connected_dfs(graph, 2, 6))
assert_equal(want=False, got=is_connected_dfs(graph, 8, 1))


## Question 2: Word Questions

How many edges are there in a **fully connected** graph with `n` nodes? That is, if you have `n` nodes, what is the maximum number of unique edges between two distinct nodes you can have?

In [ ]:
# Answer here:

What if the graph is **directed**? Then what is the maximum number of edges?

In [ ]:
# Answer here:

Why might you be able to think of an undirected graph as a special case of a directed graph?

In [ ]:
# Answer here:

#### Once done, call a TA over to check your answers!

# 3. Breadth-First Search

Breadth-First Search (BFS) is a second algorithm that can also determine whether there exists a path between two nodes `n1` and `n2`. When iterating over vertices, BFS will first touch all of the nodes with path length `1` from `n1` (i.e. `n1`'s neighbors), followed by all nodes with path length `2`, followed by path length `3`, and so on.

To achieve this, BFS uses a queue. BFS first enqueues the starting vertex `n1`. It then, until the queue is empty, repeatedly takes a node off the queue with `dequeue`, marks the node as visited, and adds all of that node's unvisited neighbors to the queue.

Suppose we wanted to run a BFS starting from node `3`. The queue at each iteration would look like the following

| Step | Queue    | Notes                                          |
|------|----------|------------------------------------------------|
| 0    | `[2]`    | Enqueue starting node                          |
| 1    | `[4]`    | Dequeue 2, enqueue 2's neighbors               |
| 2    | `[6, 8]` | Dequeue 4, enqueue 4's neighbors               |
| 3    | `[8]`    | Dequeue 6, enqueue 6's **unvisited** neighbors |
| 4    | `[]`     | Dequeue 8, enqueue 8's neighbors               |
| 5    |          | BFS terminates                                 |

### Implementation of Queues

Do not change any code here, but try to understand the different functions that a queue does

In [ ]:
def init_q(lst=None):
    """Constructs a new empty queue.

    Arguments: Optional list of initial elements in the queue (Optional[list]).
    Returns (Queue): The new empty queue.
    Effects: None.
    """
    if lst is None:
        return []
    return lst[:]


def enqueue_q(queue, elem):
    """Adds an element to the rear of the queue.

    Arguments:
        queue (Queue): The queue to which the element should be added.
        elem (Any): The element to be added to the queue.
    Returns: None.
    Effects: Modifies `queue` by adding the new element.
    """
    queue.append(elem)


def dequeue_q(queue):
    """Removes the element from the front of the queue and returns it.

    queue must not be empty.

    Arguments:
        queue (Queue): The queue from which the front element should be removed.
    Returns (any): The front element in the queue.
    Effects: The front element is removed from the queue.
    """
    return queue.pop(0)


def peek_q(queue):
    """Returns the element at the front of the queue, without removing it.

    queue must not be empty.

    Arguments:
        queue (Queue): The queue from which the front element should be returned.
    Returns (any): The front element in the queue.
    Effects: None
    """
    return queue[0]


def is_empty_q(queue):
    """Determines whether or not the queue is empty.

    Arguments:
        queue (Queue):  The queue to be checked if it is empty or not
    Returns (bool): True if the queue is empty or False if it is not empty
    Effects: None
    """
    return len(queue) == 0

Now its time for you to implement BFS. You should be using these functions in your solution
* `enqueue_q`
* `dequeue_q`
* `is_empty_q`
* `get_neighbors`


#### Pseudocode
* add the starting node `n1` to the queue
* while there are still items in the queue
    * take out the first node, `n` of the queue
    * set the first node, `n` to be visited
    * loop over all `n`'s neighbors
        * if the neighbor has not yet been visited
            * add it to the queue
* return a boolean about whether `n2` has already been visited

In [ ]:

def is_connected_bfs(G, n1, n2):
    """ returns True if there is a path from n1 to n2 in G
    Inputs:
        G: The graph
            type: list[list[int]]
        n1: A node
            type: int
        n2: Another node
            type: int
    Returns:
            type: bool
    """
    visited = [False] * len(G) # visited[i] is True if vertex i has been visited
    queue = init_q()

    # YOUR CODE HERE

In [ ]:
# Graph:
G = nx.DiGraph()
n = 10
G.add_nodes_from(range(n))
edges = []
for i in range(n - 1):
    for j in range(i + 1, n):
        if j <= i ** 2 and j % 2 == i % 2:
            edges.append((i, j))
G.add_edges_from(edges)
pos = nx.circular_layout(G)
nx.draw_networkx(G,pos=pos, arrows=True, arrowsize=10, with_labels=True, **options)
graph = nx.to_dict_of_lists(G)

assert_equal(want=False, got=is_connected_bfs(graph, 1, 4))
assert_equal(want=False, got=is_connected_bfs(graph, 7, 3))
assert_equal(want=True, got=is_connected_bfs(graph, 3, 7))
assert_equal(want=True, got=is_connected_bfs(graph, 2, 8))
assert_equal(want=True, got=is_connected_bfs(graph, 4, 6))
assert_equal(want=True, got=is_connected_bfs(graph, 2, 6))
assert_equal(want=False, got=is_connected_bfs(graph, 8, 1))

# 🤓🤓🤓🤓

### Congrats on finishing the notebook! You're doing awesome